In [1]:
import pandas as pd
import arcpy
import numpy as np
from numpy import mean
from numpy import std
import os
from arcpy import management
# from arcgis.gis import GIS
from time import time
arcpy.CheckOutExtension('Spatial')
arcpy.env.overwriteOutput = True

workspace = r"C:\Users\smcavoy\Documents\StreamsDistance_June10\\"
arcpy.env.workspace = workspace


In [2]:
#source for Bing building footprints for Indiana (attrubuted with open street maps data)
building_footprints = r"C:\Users\smcavoy\Documents\StreamsDistance_June10\BuildingFootprints_700.shp"

#set source for streams data
streams = r"C:\Users\smcavoy\Documents\StreamsDistance_June10\Indiana_50c_streams_edited.shp"

#set source for subwatersheds
all_subwatersheds = r"C:\Users\smcavoy\Documents\StreamsDistance_June10\June_6_HUCS.shp"

In [3]:
#get distance from residential buildings to streams
#project buildings and streams so they are in the same GCS
sr = arcpy.SpatialReference(4269)

arcpy.Project_management(streams, workspace + "streams_project.shp", sr)


b_copy = arcpy.CopyFeatures_management(building_footprints, workspace + "buildings_copy")


arcpy.Project_management(b_copy, workspace+"buildings_project.shp", sr)



<Result 'C:\\Users\\smcavoy\\Documents\\StreamsDistance_June10\\buildings_project.shp'>

In [4]:
#use near tool to get distance to steams
arcpy.Near_analysis(workspace+"buildings_project.shp",
                    workspace + "streams_project.shp",
                    method="GEODESIC")



<Result 'C:\\Users\\smcavoy\\Documents\\StreamsDistance_June10\\\\buildings_project.shp'>

In [5]:
subwatershed_list = [] 
dist_to_stream_avg_list = []
dist_to_stream_stdev_list = []


In [6]:
#search cursor through each row of county subwatersheds file
with arcpy.da.SearchCursor(all_subwatersheds, ['HUC12','AREA', 'PERIMETER']) as cursor:

    for row in cursor:
        subwatershed_number = row[0]
        
        subwatershed_list.append(subwatershed_number)
        
        print(subwatershed_number, " is the subwatershed to be worked on")

        #select subwatershed from shapefile with all of them
        s = arcpy.SelectLayerByAttribute_management(all_subwatersheds, "NEW_SELECTION", 
                                                "HUC12 = '{}'".format(subwatershed_number))
        
        arcpy.CopyFeatures_management(s, "s_lyr.shp")
        subwatershed_selection1 = workspace + 's_lyr.shp'
        
        arcpy.Clip_analysis(workspace+"buildings_project.shp", 
                            subwatershed_selection1,
                            f"{subwatershed_number}_buildings.shp")

        #get all the distances in to the streams
        dist_to_stream_list = []
        with arcpy.da.SearchCursor(workspace+f"{subwatershed_number}_buildings.shp", ["NEAR_DIST"]) as distCursor: 
            for dist in distCursor:  
                dist_to_stream = dist[0]  
                dist_to_stream_list.append(dist_to_stream)  

        dist_to_stream_avg = mean(dist_to_stream_list)

        dist_to_stream_stdev = std(dist_to_stream_list)

        # print(dist_to_stream_avg)
        # print(dist_to_stream_stdev)

        dist_to_stream_avg_list.append(dist_to_stream_avg)
        dist_to_stream_stdev_list.append(dist_to_stream_stdev)

051202080201  is the subwatershed to be worked on
051201110302  is the subwatershed to be worked on
051201060507  is the subwatershed to be worked on
071200010303  is the subwatershed to be worked on
050800030717  is the subwatershed to be worked on
051401010601  is the subwatershed to be worked on
051202050503  is the subwatershed to be worked on
051202030203  is the subwatershed to be worked on
051201130801  is the subwatershed to be worked on
051202060310  is the subwatershed to be worked on
051202040504  is the subwatershed to be worked on
051201040405  is the subwatershed to be worked on
071200011009  is the subwatershed to be worked on
051201010404  is the subwatershed to be worked on
041000050104  is the subwatershed to be worked on
051202030508  is the subwatershed to be worked on
051202011107  is the subwatershed to be worked on
051201100407  is the subwatershed to be worked on
051201070103  is the subwatershed to be worked on
050800030302  is the subwatershed to be worked on


051401040301  is the subwatershed to be worked on
051201090704  is the subwatershed to be worked on
050800030305  is the subwatershed to be worked on
050800030805  is the subwatershed to be worked on
051201040509  is the subwatershed to be worked on
051201060101  is the subwatershed to be worked on
051402010905  is the subwatershed to be worked on
051201060504  is the subwatershed to be worked on
051201111509  is the subwatershed to be worked on
051202030809  is the subwatershed to be worked on
051202030803  is the subwatershed to be worked on
051201111203  is the subwatershed to be worked on
050800030504  is the subwatershed to be worked on
051201110303  is the subwatershed to be worked on
050800030103  is the subwatershed to be worked on
051202050603  is the subwatershed to be worked on
051202010109  is the subwatershed to be worked on
051201040503  is the subwatershed to be worked on
051201060202  is the subwatershed to be worked on
051201111804  is the subwatershed to be worked on


051201111201  is the subwatershed to be worked on
051201070404  is the subwatershed to be worked on
051201040507  is the subwatershed to be worked on
041000030804  is the subwatershed to be worked on
051201060804  is the subwatershed to be worked on
051201060502  is the subwatershed to be worked on
051202070701  is the subwatershed to be worked on
050902030503  is the subwatershed to be worked on
051202011503  is the subwatershed to be worked on
051202040502  is the subwatershed to be worked on
051402010903  is the subwatershed to be worked on
051402011003  is the subwatershed to be worked on
051202060103  is the subwatershed to be worked on
051202040601  is the subwatershed to be worked on
051202040703  is the subwatershed to be worked on
050800030107  is the subwatershed to be worked on
050800030101  is the subwatershed to be worked on
051202011102  is the subwatershed to be worked on
051202010107  is the subwatershed to be worked on
050902031008  is the subwatershed to be worked on


051201060703  is the subwatershed to be worked on
051202080904  is the subwatershed to be worked on
051202080402  is the subwatershed to be worked on
051201040702  is the subwatershed to be worked on
051201080801  is the subwatershed to be worked on
071200020503  is the subwatershed to be worked on
051201040102  is the subwatershed to be worked on
051202080902  is the subwatershed to be worked on
051201080409  is the subwatershed to be worked on
051202070803  is the subwatershed to be worked on
051202030701  is the subwatershed to be worked on
051202010602  is the subwatershed to be worked on
040400010302  is the subwatershed to be worked on
040500010803  is the subwatershed to be worked on
051202090302  is the subwatershed to be worked on
040500011604  is the subwatershed to be worked on
051201130708  is the subwatershed to be worked on
051201100606  is the subwatershed to be worked on
051201060301  is the subwatershed to be worked on
051201130503  is the subwatershed to be worked on


041000071203  is the subwatershed to be worked on
051202060405  is the subwatershed to be worked on
051202030104  is the subwatershed to be worked on
071200011008  is the subwatershed to be worked on
051201030203  is the subwatershed to be worked on
041000040601  is the subwatershed to be worked on
051402020603  is the subwatershed to be worked on
050800030716  is the subwatershed to be worked on
051201060801  is the subwatershed to be worked on
051202020107  is the subwatershed to be worked on
051201100104  is the subwatershed to be worked on
051202070801  is the subwatershed to be worked on
051201020104  is the subwatershed to be worked on
051201080508  is the subwatershed to be worked on
051202030705  is the subwatershed to be worked on
051202040903  is the subwatershed to be worked on
051202010705  is the subwatershed to be worked on
051201010904  is the subwatershed to be worked on
051202011310  is the subwatershed to be worked on
051201080605  is the subwatershed to be worked on


In [7]:
st = time()
outputs = {'subwatershed': subwatershed_list,
           'dist_to_stream_avg (m)': dist_to_stream_avg_list,
           'dist_to_stream_stdev (m)': dist_to_stream_stdev_list}
           
outputs_df = pd.DataFrame(outputs, columns = ['subwatershed',
                                              'dist_to_stream_avg (m)',
                                              'dist_to_stream_stdev (m)'])
                                              
                                              
print(outputs_df)

outputs_df.to_excel(workspace+ '700_stream_dist.xlsx')
print(round(((time()-st)/60), 2) , 'minutes to process.')                                              

     subwatershed  dist_to_stream_avg (m)  dist_to_stream_stdev (m)
0    051202080201              884.262553                503.506692
1    051201110302             1261.467019                535.669076
2    051201060507              584.613292                297.702490
3    071200010303             1087.881755                964.682414
4    050800030717              555.426454                409.885301
..            ...                     ...                       ...
698  071200030303              802.059181                622.412597
699  051202081004             1724.139274                975.646314
700  051201111603             1036.789145                526.978627
701  051202030805              725.149980                490.462378
702  051201111502              855.611092                276.384134

[703 rows x 3 columns]
0.03 minutes to process.


In [ ]:
print(dist_to_stream_avg)
print(dist_to_stream_stdev)